#### Nous allons ici faire du webscraping afin de récupérer le contenu des articles dans les urls présents dans le fichier excel 

In [ ]:
!pip install pandas requests beautifulsoup4 newspaper3k readability-lxml openpyxl


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\stginrae.AFRIQUE-
[nltk_data]     TP02\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Parcourir les deux onglets de la feuille de calcul , ajouter des colonnes pour préciser le type (VS, NVS) et la thématique (SV, SA)


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from newspaper import Article
from readability import Document
import time
import os

# === 1. Détection automatique du bon parseur (html ou xml) ===
def get_soup(resp_text):
    if resp_text.strip().lower().startswith("<?xml") or "<rss" in resp_text.lower():
        return BeautifulSoup(resp_text, "xml")
    else:
        return BeautifulSoup(resp_text, "html.parser")

# === 2. Fonction principale d’extraction d’article ===
def extract_article_info(url):
    headers = {"User-Agent": "Mozilla/5.0"}

    # Méthode 1 : newspaper3k
    try:
        article = Article(url)
        article.download()
        article.parse()
        if len(article.text.strip().split()) > 18:
            return article.title.strip(), article.text.strip()
    except Exception:
        pass

    # Méthode 2 : readability + soup
    try:
        resp = requests.get(url, timeout=10, headers=headers)
        doc = Document(resp.text)
        soup = get_soup(doc.summary())
        content = soup.get_text(separator="\n").strip()
        if len(content.split()) > 18:
            return doc.title().strip(), content
    except Exception:
        pass

    # Méthode 3 : fallback <p>
    try:
        if 'resp' not in locals():
            resp = requests.get(url, timeout=10, headers=headers)
        soup = get_soup(resp.text)
        title = soup.title.string.strip() if soup.title else ""
        paragraphs = soup.find_all("p")
        text = "\n".join(p.get_text().strip() for p in paragraphs if p.get_text().strip())
        return title, text
    except Exception:
        return "", ""

# === 3. Scraper générique ===
def scrape_url_list(urls, type_article_label, thematique_label, is_pertinent=False):
    results = []
    excluded_domains = ["youtube.com", "youtu.be", "tiktok.com", "instagram.com", "facebook.com"]

    for url in urls:
        if not isinstance(url, str):
            continue
        if any(domain in url for domain in excluded_domains):
            print(f"URL ignorée (réseau social) : {url}")
            continue
        if url.lower().endswith(".pdf"):
            print(f"URL ignorée (PDF non traité) : {url}")
            continue

        print(f"Scraping HTML : {url}")
        title, content = extract_article_info(url)

        results.append({
            "url": url,
            "title": title,
            "content": content,
            "type_article": type_article_label,
            "thematique": thematique_label,
            "pertinent": is_pertinent
        })
        time.sleep(1)
    return results

# === 4. Scraper depuis un fichier Excel ===
def scrape_urls_from_sheet(sheet_name, type_article_label, thematique_label):
    df = pd.read_excel("data/PESVcollect_by_syndromic_keywords_S26_PversusNP.xlsx", sheet_name=sheet_name)
    url_col = next((col for col in df.columns if "url" in col.lower()), None)
    if url_col is None:
        raise ValueError("Aucune colonne URL trouvée.")
    urls = df[url_col].dropna().unique()

    # ===  définir automatiquement si pertinent ===
    is_pertinent = sheet_name == 1  # Feuille 1 -> pertinents, Feuille 0 -> non pertinents
    return scrape_url_list(urls, type_article_label, thematique_label, is_pertinent=is_pertinent)

# === 5. Liste des URLs manuelles (pertinentes) ===
manual_urls = [
    "https://dailypost.ng/2025/02/13/kebbi-govt-battles-mysterious-disease-affecting-rice-farms/",
    "https://www.martinique.franceantilles.fr/actualite/fil-info/en-guyane-le-manioc-decime-par-une-maladie-inconnue-973463.php",
    "https://www.monitor.co.ug/uganda/news/national/strange-disease-attacks-cassava-gardens-in-nebbi-3217576",
    "https://thestarzambia.wordpress.com/2020/11/20/fungal-disease-destroys-cassava-fields-in-manyinga/"
]

# === 6. Calcul du nombre de mots ===
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

# === 7. Scraping principal ===
results_nsv = scrape_urls_from_sheet(sheet_name=0, type_article_label="NVS", thematique_label="SV")
results_vs = scrape_urls_from_sheet(sheet_name=1, type_article_label="VS", thematique_label="SV")
results_manual = scrape_url_list(manual_urls, type_article_label="VS", thematique_label="SV", is_pertinent=True)

# === 8. Fusion et nettoyage ===
all_results = results_nsv + results_vs + results_manual
df_result = pd.DataFrame(all_results)
df_result["word_count"] = df_result["content"].apply(count_words)
df_result = df_result[df_result["word_count"] >= 18] # On prend les articles qui ont au moins 18 mots

# === 9. Export final ===
df_result.to_excel("articles_scraped_final.xlsx", index=False)
print("Scraping complet terminé")


Scraping HTML : https://www.mdpi.com/2073-4395/15/7/1551
Scraping HTML : https://www.glassdoor.com/job-listing/plant-health-care-specialist-savatree-JV_IC1132183_KO0,28_KE29,37.htm?jl=1009770976017
Scraping HTML : https://gns3.com/community/discussions/the-hidden-secret-to-thriving-with-my-survival-farm-what-youre-missing
Scraping HTML : https://www.bbc.co.uk/news/articles/cy9xv90lg7do
Scraping HTML : https://www.sciencedirect.com/science/article/pii/S095741742502411X
Scraping HTML : https://www.michiganfarmnews.com/downy-mildew-confirmed-in-mi-cucumbers-as-spore-spread-continues
URL ignorée (réseau social) : https://www.youtube.com/watch?v=4CgtU9rnvNE
Scraping HTML : https://www.bbc.co.uk/news/articles/czryvm3nlvdo
Scraping HTML : https://en.wikipedia.org/wiki/Beech_leaf_disease
Scraping HTML : https://www.detroitnews.com/story/news/local/michigan/2025/06/28/plant-experts-question-risk-agroterrorism-claim-chinese-lab-tech-case-university-michigan/84263101007/
URL ignorée (réseau socia

### Nous allons faire une vérification des textes collectés

Nous détecterons que la langue est bien l'angais, nous détecterons les articles qui ont des caractères incompréhensibles en caculant les proportions 

In [4]:
import re
import unicodedata
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 0  # Pour rendre les résultats reproductibles

# === Détection de la langue avec fallback ===
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# === Proportion de caractères "bruités" (non alphanumériques et non ponctuation simple) ===
def get_noise_ratio(text):
    if not isinstance(text, str) or not text.strip():
        return 1.0

    # Normalisation unicode
    text = unicodedata.normalize("NFKC", text)

    # On enlève les espaces, sauts de ligne et tabulations
    cleaned = re.sub(r"\s+", "", text)

    if len(cleaned) == 0:
        return 1.0

    # Caractères "normaux" = lettres latines, chiffres, ponctuations usuelles
    clean_chars = re.findall(r"[a-zA-Z0-9.,;:!?\'\"()\[\]-]", cleaned)
    ratio_clean = len(clean_chars) / len(cleaned)

    # On retourne le ratio de caractères "bruités"
    return 1.0 - ratio_clean


Appliquons ce prétraitement à nos textes 

In [5]:
print("> Détection de la langue...")
df_result["langue"] = df_result["content"].apply(detect_language)

print("> Calcul du taux de bruit (caractères bizarres)...")
df_result["pct_bizarre"] = df_result["content"].apply(get_noise_ratio)

# === 4. Filtrage : contenu anglais, lisible, assez long ===
df_result_filtered = df_result[
    (df_result["langue"] == "en") &
    (df_result["pct_bizarre"] < 0.3) &
    (df_result["word_count"] >= 18)
].copy()

# === 5. Export des résultats ===
df_result.to_excel("articles_scraped_all.xlsx", index=False)
df_result_filtered.to_excel("articles_scraped_filtered.xlsx", index=False)

print("> Export terminé :")
print("> articles_scraped_all.xlsx (complet)")
print("> articles_scraped_filtered.xlsx (filtré)")


> Détection de la langue...
> Calcul du taux de bruit (caractères bizarres)...
> Export terminé :
> articles_scraped_all.xlsx (complet)
> articles_scraped_filtered.xlsx (filtré)


In [6]:
df_result["langue"].value_counts()

langue
en    373
fr      7
bn      1
de      1
da      1
Name: count, dtype: int64

In [7]:
data = pd.read_excel("articles_scraped_filtered.xlsx")

data.head()

,url,title,content,type_article,thematique,pertinent,word_count,langue,pct_bizarre
0,https://www.mdpi.com/2073-4395/15/7/1551,Utilizing Environmentally Friendly Techniques ...,The review aims to provide a comprehensive ana...,NVS,SV,False,10956,en,0.001252
1,https://gns3.com/community/discussions/the-hid...,[no-title],Incompatible Browser!\nLooks like you're using...,NVS,SV,False,19,en,0.000000
2,https://www.bbc.co.uk/news/articles/cy9xv90lg7do,Diseased ash trees in Redditch to be felled to...,Trees infected with a deadly disease in Reddit...,NVS,SV,False,107,en,0.000000
3,https://www.sciencedirect.com/science/article/...,ScienceDirect,There was a problem providing the content you ...,NVS,SV,False,29,en,0.014706
4,https://www.michiganfarmnews.com/downy-mildew-...,Access denied | www.michiganfarmnews.com used ...,The owner of this website (www.michiganfarmnew...,NVS,SV,False,45,en,0.031073


In [8]:
urls = data["url"].tolist()

url = [  "https://dailypost.ng/2025/02/13/kebbi-govt-battles-mysterious-disease-affecting-rice-farms/",
    "https://www.martinique.franceantilles.fr/actualite/fil-info/en-guyane-le-manioc-decime-par-une-maladie-inconnue-973463.php",
    "https://www.monitor.co.ug/uganda/news/national/strange-disease-attacks-cassava-gardens-in-nebbi-3217576",
    "https://thestarzambia.wordpress.com/2020/11/20/fungal-disease-destroys-cassava-fields-in-manyinga/"]

for u in urls:
    if u in url:
        print("yes")

yes
yes
yes


Cet article n'a pas été pris en compte, en effet c'est un article qui est en français 

"https://www.martinique.franceantilles.fr/actualite/fil-info/en-guyane-le-manioc-decime-par-une-maladie-inconnue-973463.php"

In [128]:
data["type_article"].value_counts()

type_article
NVS    348
VS       7
Name: count, dtype: int64

In [129]:
data["word_count"].mean()

np.float64(991.6450704225352)

En moyenne les textes ont une longeur de 932 mots 

### Voir les articles qui ont des caractères incompréhensibles 

In [9]:
mauvais = df_result[df_result["pct_bizarre"]>=0.3]

In [10]:
mauvais

,url,title,content,type_article,thematique,pertinent,word_count,langue,pct_bizarre
294,https://dergipark.org.tr/tr/download/article-f...,[no-title],%PDF-1.7\n%����\n2 0 obj\n<<\n/Lang (tr-TR)\n/...,NVS,SV,False,117,en,0.347122
341,https://www.iowadnr.gov/media/8460/download?in...,[no-title],%PDF-1.7\r\n%����\r\n1 0 obj\r\n<>/Metadata 20...,NVS,SV,False,180,bn,0.627653


In [121]:
mauvais.iloc[0]["url"]

'https://dergipark.org.tr/tr/download/article-file/4410662'

In [132]:
df_result["type_article"].value_counts()

type_article
NVS    359
VS       8
Name: count, dtype: int64

In [11]:
# Statistiques descriptives de la longueur des articles
word_counts = data["word_count"].dropna()

mean_wc = word_counts.mean()
median_wc = word_counts.median()
q1 = word_counts.quantile(0.25)
q3 = word_counts.quantile(0.75)
min_wc = word_counts.min()
max_wc = word_counts.max()
iqr = q3 - q1

print("Statistiques sur la longueur des articles (en nombre de mots) :")
print(f"- Moyenne     : {mean_wc:.2f}")
print(f"- Médiane     : {median_wc}")
print(f"- 1er quartile (Q1) : {q1}")
print(f"- 3e quartile (Q3) : {q3}")
print(f"- Écart interquartile (IQR) : {iqr}")
print(f"- Min         : {min_wc}")
print(f"- Max         : {max_wc}")


Statistiques sur la longueur des articles (en nombre de mots) :
- Moyenne     : 830.24
- Médiane     : 334.0
- 1er quartile (Q1) : 67.0
- 3e quartile (Q3) : 797.25
- Écart interquartile (IQR) : 730.25
- Min         : 19
- Max         : 15029


## Résumé:
Nous avons: 
- Parcouru les urls dans le fichier excel, dans les deux onglets plus précisément, le premier onglet pour les articles non pertinents et le second pour ceux pertinents
- Ajouté les urls reçus par mail, qui sont d'ailleur considérés comme pertinent 
- Filtré les articles avec peu de caractères, nous avons pris les articles qui ont au moins 51 mots 
- Supprimé les articles qui ne sont pas en anglais en détectant la langue avec langdetect 
- Supprimé les articles avec les caractères non compréhensibles en calculant une proportion de caractères bizarre, si supérieur à 0.3 alors cet article est considéré comme incompréhensible. Nous avons vérifié le contenu de ces articles et il ne disait rien de bon effectivement 
- Calculé le nombre de mots dans les articles finaux, fait des statistiques afin de savoir comment sont les articles dans l'ensemble 
- Avec ceci on peut déjà constituer notre jeu de données 

### Tous ces articles parlent de la thématique santé végétale 

Maintenant nous allons attaquer d'autres fichiers csv qui contiennent plusieurs thématiques 


In [12]:
new_data1 = pd.read_csv("data/unknown_rssfeed_2021.csv", sep=";")

new_data1.head()

,id,title,label,url,source,text,lang,probability_lang,date_submitted,date_aspirated,...,source_lang,from_direct_url,is_reference,reference_id,reference_key,external_id,continent,country,is_archived,is_processed
0,011XWKNNSM,NSW Covid outbreak: death toll rises as two-th...,H_disease,https://www.theguardian.com/australia-news/202...,www.theguardian.com,The death toll from the New South Wales corona...,EN,0.999998,1627713480,1627810577,...,NaN,0,0,NaN,NaN,NaN,NaN,US,0,0
1,027EFC5QA1,Doctors in Pakistan baffled by mystery dengue-...,H_unknown,https://www.telegraph.co.uk/global-health/scie...,www.telegraph.co.uk,"Dr Zeeshan Hussain, senior hematopathologist a...",EN,0.999997,1637067600,1637130613,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0
2,05EARH16XI,Nationwide salmonella outbreak linked to fresh...,H_disease,https://www.denverpost.com/2021/10/21/onion-sa...,www.denverpost.com,Your onions may do more than make you cry when...,EN,0.999996,1634821696,1634834672,...,NaN,0,0,NaN,NaN,NaN,NaN,US,0,0
3,06HNI2YAI3,"Victoria records 55 local COVID-19 cases, with...",H_disease,https://www.abc.net.au/news/2021-08-20/victori...,www.abc.net.au,Victorians will know today if there will be a ...,EN,0.999997,1629413579,1629508346,...,NaN,0,0,NaN,NaN,NaN,OC,AU,0,0
4,07J1WGYB3J,"After UP’s Firozabad, mystery fever claims liv...",H_unknown,https://www.indiatoday.in/india/story/ystery-f...,www.indiatoday.in,A mystery fever has claimed the lives of eight...,EN,0.999998,1631453480,1631539804,...,NaN,0,0,NaN,NaN,NaN,AS,IN,0,0


In [55]:
new_data1.shape

(419, 29)

In [56]:
new_data1["label"].value_counts()

label
H_disease            243
H_unknown             95
A_unknown             52
A_avian_influenza      6
V_unknown              5
A_suspicion            4
A_CWD                  4
A_rabies               3
IRR                    2
A_fungus               2
A_coronavirus          1
A_AHS                  1
A_EHV                  1
Name: count, dtype: int64

In [50]:
new_data2 = pd.read_csv("data/unknown_rssfeed_2022.csv", sep=";")

new_data2.head()

,id,title,label,url,source,text,lang,probability_lang,date_submitted,date_aspirated,...,source_lang,from_direct_url,is_reference,reference_id,reference_key,external_id,continent,country,is_archived,is_processed
0,01KWR761C8,Limpopo records second measles outbreak - NICD,H_disease,https://news.google.com/__i/rss/rd/articles/CB...,news.google.com,Limpopo records second measles outbreak - NICD...,EN,0.999997,1667821286,1667830172,...,NaN,0,0,NaN,NaN,NaN,AF,ZA,0,0
1,0PLRZC18KB,CDC Investigation Notice Update: CDC Updates N...,H_disease,https://www.cdc.gov/media/releases/2022/s-0826...,www.cdc.gov,CDC Investigation Notice Update: CDC Updates N...,EN,0.999996,1660888800,1662346579,...,NaN,0,0,NaN,NaN,NaN,NaN,US,0,0
2,0YJNMFSF06,"New mysterious disease spread in Argentina, mo...",H_unknown,https://huntdailynews.in/new-mysterious-diseas...,huntdailynews.in,"New mysterious disease spread in Argentina, mo...",EN,0.999997,1662907625,1662978605,...,NaN,0,0,NaN,NaN,NaN,AS,IN,0,0
3,11TK7SONPZ,Measles cases rise to 169 in South Africa outb...,H_disease,https://news.google.com/__i/rss/rd/articles/CB...,news.google.com,Measles cases rise to 169 in South Africa outb...,EN,0.999998,1671306255,1671407185,...,NaN,0,0,NaN,NaN,NaN,AF,ZA,0,0
4,13XHLD2N77,Madhya Pradesh: Unidentified ailment claims li...,H_unknown,https://news.google.com/__i/rss/rd/articles/CB...,news.google.com,Madhya Pradesh: Unidentified ailment claims li...,EN,0.999998,1671724360,1671736139,...,NaN,0,0,NaN,NaN,NaN,AS,IN,0,0


In [58]:
new_data2.shape

(223, 29)

In [59]:
new_data2["label"].value_counts()

label
H_disease               128
H_unknown                54
A_unknown                14
V_unknown                 5
A_avian_influenza         5
irrelevant                5
other                     3
A_rabies                  2
A_CWD                     2
V_xylella_fastidiosa      1
A_distemper               1
A_cholera                 1
A_RHD                     1
A_parvovirus              1
Name: count, dtype: int64

In [51]:
new_data3 = pd.read_csv("data/unknown_rssfeed_2023.csv", sep=";")

new_data3.head()

,id,title,label,url,source,text,lang,probability_lang,date_submitted,date_aspirated,...,source_lang,from_direct_url,is_reference,reference_id,reference_key,external_id,continent,country,is_archived,is_processed
0,04N6IMJ5WG,"China sees average 7,000 pneumonia cases amid ...",H_unknown,https://www.hindustantimes.com/world-news/chin...,www.hindustantimes.com,"China sees average 7,000 pneumonia cases amid ...",EN,0.999997,1700722800,1709089092,...,NaN,0,0,NaN,NaN,https://news.google.com/rss/articles/CBMilAFod...,AS,CN,0,1
1,0G4V0IWCNX,FDA reports two new outbreaks of unknown origin,H_unknown,https://www.foodsafetynews.com/2023/11/fda-rep...,www.foodsafetynews.com,FDA reports two new outbreaks of unknown origi...,EN,0.999997,1700107692,1700859601,...,NaN,0,0,NaN,NaN,https://news.google.com/rss/articles/CBMiV2h0d...,NaN,NaN,0,1
2,1AFCNN4Y56,Mysterious Respiratory Illness Affecting Dogs ...,A_unknown,https://timesofsandiego.com/life/2023/11/29/my...,timesofsandiego.com,Mysterious Respiratory Illness Affecting Dogs ...,EN,0.999998,1701241200,1702732121,...,NaN,0,0,NaN,NaN,https://news.google.com/rss/articles/CBMikQFod...,NaN,US,0,1
3,1MU92KZ24Z,African Health Authorities Juggle Concurrent O...,H_unknown,https://news.google.com/__i/rss/rd/articles/CB...,news.google.com,African Health Authorities Juggle Concurrent O...,EN,0.999996,1674749451,1674788005,...,NaN,0,0,NaN,NaN,NaN,AF,NG,0,0
4,1QOSF6VHC5,Mystery illness affecting thousands of dogs. S...,A_unknown,https://fortworthreport.org/2023/12/16/mystery...,fortworthreport.org,Mystery illness affecting thousands of dogs. S...,EN,0.999996,1702760400,1702829625,...,NaN,0,0,NaN,NaN,https://news.google.com/rss/articles/CBMifmh0d...,NaN,US,0,1


In [61]:
new_data3["label"].value_counts()

label
H_unknown                 46
A_unknown                 40
H_disease                 39
irrelevant                 4
A_avian_influenza          3
A_rabies                   2
V_unknown                  1
A_EEE                      1
A_newcastle                1
A_vesicular_stomatitis     1
A_mad_cow                  1
A_CWD                      1
A_west_nile                1
A_Echinococcus             1
Name: count, dtype: int64

#### Nous allons rassembler ces articles sous la forme des articles scrapés afin de créer un jeu de données final pour le moment 

In [13]:
import pandas as pd
import re
from langdetect import detect

# === 1. Chargement des fichiers annotés ===
df_2021 = pd.read_csv("data/unknown_rssfeed_2021.csv", sep=";")
df_2022 = pd.read_csv("data/unknown_rssfeed_2022.csv", sep=";")
df_2023 = pd.read_csv("data/unknown_rssfeed_2023.csv", sep=";")

annotated_df = pd.concat([df_2021, df_2022, df_2023], ignore_index=True)
annotated_df.columns = [col.lower().strip() for col in annotated_df.columns]

# === 2. Nettoyage initial
irrelevant_labels = ["irrelevant", "other", "irr"]
annotated_df["label"] = annotated_df["label"].str.lower().str.strip()
annotated_df["pertinent"] = ~annotated_df["label"].isin(irrelevant_labels)

# === 3. Attribution des champs type_article et thematique ===
def map_labels_to_type_thematique(label):
    if label in irrelevant_labels:
        # NVS mais on déduit quand même la thématique par le préfixe
        if label.startswith("a_"):
            return "NVS", "SA"
        elif label.startswith("h_"):
            return "NVS", "SP"
        elif label.startswith("v_"):
            return "NVS", "SV"
        else:
            return "NVS", "Inconnu"
    else:
        # VS pour les articles de la veille syndromique on regarde aussi la thématique 
        if label.startswith("a_"):
            return "VS", "SA"
        elif label.startswith("h_"):
            return "VS", "SP"
        elif label.startswith("v_"):
            return "VS", "SV"
        else:
            return "VS", "Inconnu"


annotated_df[["type_article", "thematique"]] = annotated_df["label"].apply(
    lambda lbl: pd.Series(map_labels_to_type_thematique(lbl))
)

# === 4. Ajout des colonnes d'analyse texte ===

# Nombre de mots
annotated_df["word_count"] = annotated_df["text"].apply(lambda x: len(str(x).split()) if isinstance(x, str) else 0)

# Langue
DetectorFactory.seed = 0  # Pour rendre les résultats reproductibles

# === Détection de la langue avec fallback ===
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# === Proportion de caractères "bruités" (non alphanumériques et non ponctuation simple) ===
def get_noise_ratio(text):
    if not isinstance(text, str) or not text.strip():
        return 1.0

    # Normalisation unicode
    text = unicodedata.normalize("NFKC", text)

    # On enlève les espaces, sauts de ligne et tabulations
    cleaned = re.sub(r"\s+", "", text)

    if len(cleaned) == 0:
        return 1.0

    # Caractères "normaux" = lettres latines, chiffres, ponctuations usuelles
    clean_chars = re.findall(r"[a-zA-Z0-9.,;:!?\'\"()\[\]-]", cleaned)
    ratio_clean = len(clean_chars) / len(cleaned)

    # On retourne le ratio de caractères "bruités"
    return 1.0 - ratio_clean

annotated_df["noise_ratio"] = annotated_df["text"].apply(get_noise_ratio)
annotated_df["lang"] = annotated_df["text"].apply(detect_language)

# === 5. Filtrage qualité des articles ===
annotated_df = annotated_df[
    (annotated_df["word_count"] >= 18) & # Nous prenons 18 car c'est la plus petite taille des articles de PADI-WEB 
    (annotated_df["lang"] == "en") &
    (annotated_df["noise_ratio"] < 0.3)
].copy()

# === 6. Ajout de colonne source
annotated_df["source"] = "flux_unknown"


In [14]:
annotated_df.head()

,id,title,label,url,source,text,lang,probability_lang,date_submitted,date_aspirated,...,external_id,continent,country,is_archived,is_processed,pertinent,type_article,thematique,word_count,noise_ratio
0,011XWKNNSM,NSW Covid outbreak: death toll rises as two-th...,h_disease,https://www.theguardian.com/australia-news/202...,flux_unknown,The death toll from the New South Wales corona...,en,0.999998,1627713480,1627810577,...,NaN,NaN,US,0,0,True,VS,SP,644,0.006873
1,027EFC5QA1,Doctors in Pakistan baffled by mystery dengue-...,h_unknown,https://www.telegraph.co.uk/global-health/scie...,flux_unknown,"Dr Zeeshan Hussain, senior hematopathologist a...",en,0.999997,1637067600,1637130613,...,NaN,NaN,NaN,0,0,True,VS,SP,245,0.009228
2,05EARH16XI,Nationwide salmonella outbreak linked to fresh...,h_disease,https://www.denverpost.com/2021/10/21/onion-sa...,flux_unknown,Your onions may do more than make you cry when...,en,0.999996,1634821696,1634834672,...,NaN,NaN,US,0,0,True,VS,SP,360,0.001159
3,06HNI2YAI3,"Victoria records 55 local COVID-19 cases, with...",h_disease,https://www.abc.net.au/news/2021-08-20/victori...,flux_unknown,Victorians will know today if there will be a ...,en,0.999997,1629413579,1629508346,...,NaN,OC,AU,0,0,True,VS,SP,1789,0.000106
4,07J1WGYB3J,"After UP’s Firozabad, mystery fever claims liv...",h_unknown,https://www.indiatoday.in/india/story/ystery-f...,flux_unknown,A mystery fever has claimed the lives of eight...,en,0.999998,1631453480,1631539804,...,NaN,AS,IN,0,0,True,VS,SP,461,0.004318


In [15]:
annotated_df["pertinent"].value_counts()

pertinent
True     766
False     14
Name: count, dtype: int64

In [5]:
annotated_df.columns

Index(['id', 'title', 'label', 'url', 'source', 'text', 'lang',
       'probability_lang', 'date_submitted', 'date_aspirated', 'id_rssfeed',
       'source_text', 'source_title', 'description', 'processed_text',
       'created_at', 'published_at', 'content_type', 'extension',
       'source_lang', 'from_direct_url', 'is_reference', 'reference_id',
       'reference_key', 'external_id', 'continent', 'country', 'is_archived',
       'is_processed', 'pertinent', 'type_article', 'thematique', 'word_count',
       'noise_ratio'],
      dtype='object')

In [16]:
new_data = annotated_df[["url","title", "text", "type_article", "thematique", "pertinent", "word_count"]]

In [17]:
data.rename(columns={"content" : "text"}, inplace=True)

In [18]:
data = data[["url","title", "text", "type_article", "thematique", "pertinent", "word_count"]]

In [19]:
data_final = pd.concat([data, new_data], ignore_index=True)

In [20]:
data_final.shape

(1152, 7)

In [21]:
data_final.to_csv("data/aticles_pertinent_non_pertinent.csv", index=False)

In [22]:
df = pd.read_csv("data/aticles_pertinent_non_pertinent.csv")

df.head()

,url,title,text,type_article,thematique,pertinent,word_count
0,https://www.mdpi.com/2073-4395/15/7/1551,Utilizing Environmentally Friendly Techniques ...,The review aims to provide a comprehensive ana...,NVS,SV,False,10956
1,https://gns3.com/community/discussions/the-hid...,[no-title],Incompatible Browser!\nLooks like you're using...,NVS,SV,False,19
2,https://www.bbc.co.uk/news/articles/cy9xv90lg7do,Diseased ash trees in Redditch to be felled to...,Trees infected with a deadly disease in Reddit...,NVS,SV,False,107
3,https://www.sciencedirect.com/science/article/...,ScienceDirect,There was a problem providing the content you ...,NVS,SV,False,29
4,https://www.michiganfarmnews.com/downy-mildew-...,Access denied | www.michiganfarmnews.com used ...,The owner of this website (www.michiganfarmnew...,NVS,SV,False,45


In [23]:
df["thematique"].value_counts()

thematique
SP         601
SV         384
SA         153
Inconnu     14
Name: count, dtype: int64

In [24]:
df["pertinent"].value_counts()

pertinent
True     773
False    379
Name: count, dtype: int64

In [25]:
df["type_article"].value_counts()

type_article
VS     773
NVS    379
Name: count, dtype: int64

#### Commentaire:
Nous avons bien une correspondance entre le champ pertinent et type_article c'est plus cohérent maintenant

### Suite:
Nous allons appliquer la classification binaire avec ces articles que nous avons actuellement, mais essayons de faire un peu de statistiques sur le nombre de mots des articles que nous avons sauvegardés

In [26]:
# Statistiques descriptives de la longueur des articles
word_counts = df["word_count"].dropna()

mean_wc = word_counts.mean()
median_wc = word_counts.median()
q1 = word_counts.quantile(0.25)
q3 = word_counts.quantile(0.75)
min_wc = word_counts.min()
max_wc = word_counts.max()
iqr = q3 - q1

print("Statistiques sur la longueur des articles (en nombre de mots) :")
print(f"- Moyenne     : {mean_wc:.2f}")
print(f"- Médiane     : {median_wc}")
print(f"- 1er quartile (Q1) : {q1}")
print(f"- 3e quartile (Q3) : {q3}")
print(f"- Écart interquartile (IQR) : {iqr}")
print(f"- Min         : {min_wc}")
print(f"- Max         : {max_wc}")


Statistiques sur la longueur des articles (en nombre de mots) :
- Moyenne     : 684.54
- Médiane     : 444.0
- 1er quartile (Q1) : 260.75
- 3e quartile (Q3) : 737.25
- Écart interquartile (IQR) : 476.5
- Min         : 19
- Max         : 15029


### Nous allons encore ajouter les données en combinant avec ce que nous avons reçues de PADI-WEB 

In [27]:
## Essayons d'ouvrir un fichier afin de voir la structure du dataframe

data = pd.read_csv("data/syndromic_surveillance/sv_articles_syndromic_surveillance_2020_irrelevant_user.csv")

data.shape

(15, 27)

#### Nous allons charger et préparer ces données comme avec le pipeline précédent sur la deuxième source des données 

In [28]:
import pandas as pd
import re
import os
import unicodedata
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 0

nb = 0

# === Détection langue robuste ===
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# === Taux de bruit ===
def get_noise_ratio(text):
    if not isinstance(text, str) or not text.strip():
        return 1.0
    text = unicodedata.normalize("NFKC", text)
    cleaned = re.sub(r"\s+", "", text)
    if len(cleaned) == 0:
        return 1.0
    clean_chars = re.findall(r"[a-zA-Z0-9.,;:!?\'\"()\[\]-]", cleaned)
    return 1.0 - len(clean_chars) / len(cleaned)

# === Récupération de tous les fichiers CSV ===
folder = "data/syndromic_surveillance"
files = [f for f in os.listdir(folder) if f.endswith(".csv")]

all_data = []

for file in files:
    path = os.path.join(folder, file)
    df = pd.read_csv(path)
    print(df.shape)
    nb+= df.shape[0]

    # Normaliser les noms de colonnes
    df.columns = [col.lower().strip() for col in df.columns]

    # Ajouter pertinence et type_article
    is_pertinent = "relevant" in file
    df["pertinent"] = is_pertinent
    df["type_article"] = "VS" if is_pertinent else "NVS"

    # Forcer la thématique à "SV"
    df["thematique"] = "SV"

    # Extraire l'année depuis le nom de fichier
    year_match = re.search(r"20\d{2}", file)
    df["year"] = int(year_match.group()) if year_match else None

    all_data.append(df)

# Fusionner
df_surveillance = pd.concat(all_data, ignore_index=True)

# Nettoyage
df_surveillance["text"] = df_surveillance["text"].astype(str)
df_surveillance["word_count"] = df_surveillance["text"].apply(lambda x: len(x.split()))
df_surveillance["lang"] = df_surveillance["text"].apply(detect_language)
df_surveillance["noise_ratio"] = df_surveillance["text"].apply(get_noise_ratio)
df_surveillance["source"] = "syndromic_surveillance"

# Filtrage
df_surveillance_clean = df_surveillance[
    (df_surveillance["word_count"] >= 18) &
    (df_surveillance["lang"] == "en") &
    (df_surveillance["noise_ratio"] < 0.3)
].copy()


(1, 27)
(1, 27)
(1, 27)
(5, 27)
(1, 17)
(15, 27)
(1, 17)
(3, 27)
(2, 17)
(9, 27)
(1, 17)
(9, 27)
(4, 17)
(9, 27)
(1, 17)
(7, 27)
(7, 17)


In [29]:
df_surveillance_clean.columns

Index(['id', 'title', 'url', 'text', 'reference_key', 'reference_id',
       'is_reference', 'keyword_list', 'location_list', 'publication_date',
       'rssfeed_id', 'source', 'country', 'continent', 'lang', 'source_lang',
       'created_at', 'cls_relevance', 'cls_surveillance type',
       'cls_reglementation', 'cls_health reports and interceptions',
       'cls_surveillance', 'cls_control measures', 'cls_communication',
       'cls_epidemiological, socio-economic and environmental risk',
       'cls_methods of analysis and detection',
       'cls_genetic and molecular scale', 'pertinent', 'type_article',
       'thematique', 'year', 'word_count', 'noise_ratio'],
      dtype='object')

In [30]:
df1 = pd.read_csv("data/aticles_pertinent_non_pertinent.csv")

df1.columns

Index(['url', 'title', 'text', 'type_article', 'thematique', 'pertinent',
       'word_count'],
      dtype='object')

In [31]:
df2 = df_surveillance_clean[['url', 'title', 'text', 'type_article', 'thematique', 'pertinent',
       'word_count']]
df2.shape

(77, 7)

In [32]:
df = pd.concat([df1,df2], ignore_index=True)

df.shape

(1229, 7)

In [33]:
df["type_article"].value_counts()

type_article
VS     850
NVS    379
Name: count, dtype: int64

In [ ]:
# Va écraser l'ancien qui ne contenait pas les données de PADI-WEB 

df.to_csv("data/articles_pertinent_non_pertinent.csv", index=False)

In [2]:
import pandas as pd 

data = pd.read_csv("data/aticles_pertinent_non_pertinent.csv", keep_default_na=False)

data.shape

(1229, 7)

In [3]:
# Suppression des lignes avec thématique "SP" ou "Inconnu"
data_filtered = data[~data["thematique"].isin(["SP", "Inconnu"])]


C'est ici que nous avons fait la sauvegarde du fichier *articles_SA&SV.csv*

In [5]:
data_filtered["type_article"].value_counts()

type_article
NVS    365
VS     249
Name: count, dtype: int64

voyons si cela confirme bien nos doutes

In [6]:
data_filtered[(data_filtered["type_article"] =="NVS") & (data_filtered["thematique"] == "SA")]

,url,title,text,type_article,thematique,pertinent,word_count


Nous voyons bien que les articles qui sont à la fois NVS et de SA il y en a pas. Nous allons donc continuer avec l'ajout des données de la phase 1 tout en prenant les textes dans lesquels nous n'avions pas supprimé les noms de maladie 

## Définition des données

Le fichier *articles_SA&SV.csv* contient les articles nouvellement collectés pour la phase 2 du stage qui rassemble à la fois les articles de padiweb, ceux de ESV et ESA. Dans ces articles il n'y a pas eu suppression des noms de maladie, nous avons sélectionné uniquement les thématique SA et SV pour le constituer 

Dans le fichier *aticles_pertinent_non_pertinent.csv* nous avons SA, SP, et SV comme thématique pourtant nous voulons uniquement SA et SV pour le moment, c'est à partir de ça que nous avons formé le fichier *articles_SA&SV.csv* 

In [1]:
import pandas as pd 
data1 = pd.read_csv("data/articles_SA&SV.csv", keep_default_na=False)

data2 = pd.read_csv("data/articles_selectionnes_umap_et_kmeans.csv", keep_default_na=False)


Nous allons concatener data1 et data2 puis faire un filtrage des articles de la thématique SA et SV ensuite former les données finaux qui aurons la répartition 1/10 et 9/10 sur type_article 

In [140]:
data2.columns

Index(['title', 'text', 'thematique', 'Maladie', 'text_sans_html',
       'text_concat', 'clean_text', 'x', 'y', 'z', 'cluster', 'dist'],
      dtype='object')

In [2]:
data1.columns

Index(['url', 'title', 'text', 'type_article', 'thematique', 'pertinent',
       'word_count'],
      dtype='object')

### On va former 3 jeux de données 
- articles avec noms de maladie de phase 1  et ceux de phase 2 (avec nom de maladie)
- articles sans noms de maladie de phase 1 (clean_text) et ceux avec noms de maladie de phase 2 ( c'est avec ce jeu de données qu'on avait de meilleur performance TF-IDF + XGBOOST)
- artilces sans noms de maladie de phase 1 et phase 2

In [ ]:
data2_1 = data2[["text_concat", "thematique"]]

data2_1.rename(columns={"text_concat":"text"}, inplace=True)

data2_1.head()

# Avec ceci on forme le premier jeu de données 

C:\Users\stginrae.AFRIQUE-TP02\AppData\Local\Temp\ipykernel_18084\3257169933.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2_1.rename(columns={"text_concat":"text"}, inplace=True)


,text,thematique
0,Bani Suef emergency in preparation for spring ...,SA
1,Preventive vaccination campaign against foot-a...,SA
2,Janati: The herd's condition is good. And Onsa...,SA
3,As Eid al-Adha approaches. Onsa manager reveal...,SA
4,Intensive clean-up campaigns in assiut neighbo...,SA


*text* ici contient les articles nettoyées de la phase 1 (suppression des noms de maladie, suppression des caractères incompréhensibles, mais pas de tokenisation, lemmatisation...)

Nous allons faire un mélange de ces données avec les nouvelles données de data1 puis faire le prétraitement sans supprimer les noms de maladie et former la première version de texte nettoyé, qu'on pourra toujours testé à titre de comparaison, on formera un jeu de données à par pour cela 

In [168]:
data2_2 = data2[["clean_text", "thematique"]]

data2_2.rename(columns={"clean_text":"text"}, inplace=True)

data2_2.head()

# Avec ceci on forme le premier jeu de données 

C:\Users\stginrae.AFRIQUE-TP02\AppData\Local\Temp\ipykernel_18084\4103866948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2_2.rename(columns={"clean_text":"text"}, inplace=True)


,text,thematique
0,Bani Suef emergency in preparation for spring ...,SA
1,Preventive vaccination campaign against foot-a...,SA
2,Janati: The herd's condition is good. And Onsa...,SA
3,As Eid al-Adha approaches. Onsa manager reveal...,SA
4,Intensive clean-up campaigns in assiut neighbo...,SA


*text* ici contient les articles de la phase 1 ayant subit la suppression des noms de maladie et ceux de la phase 2 contenant encore les noms de maladie: deuxième version du jeu de données pour cette expérience 

### Pour former la troisième version de données, nous allons simplement prendre les données de la version 1 et faire le pré_nettoyage (suppression des noms de maladie) et le nettoyage (lemmatisation,..)

In [163]:
### Concatener title et text de data1, dans data2 on avait déjà fait cela 

data1["text"] = data["title"] + " " + data["text"]

data1 = data1[["text", "type_article", "thematique"]]

In [3]:
import pandas as pd

# 1. Marquer les anciennes données (celles de la phase 1) comme NVS
data2["type_article"] = "NVS"

# 2. Colonnes d’intérêt
columns = ["text", "type_article", "thematique"]

# 3. Assurer les mêmes colonnes
data1 = data1[columns]
data2 = data2[columns]

# 4. Fusion complète des deux jeux
full_data = pd.concat([data1, data2], ignore_index=True)

# 5. Ne garder que les articles dont la thématique est SA ou SV
filtered_data = full_data[full_data["thematique"].isin(["SA", "SV"])].copy()

# 6. Séparer VS et NVS dans ce sous-ensemble
vs_data = filtered_data[filtered_data["type_article"] == "VS"]
nvs_data = filtered_data[filtered_data["type_article"] == "NVS"]

# 7. Calcul du nombre nécessaire de NVS pour avoir 4/5 de l'ensemble
n_vs = len(vs_data)
n_nvs_target = 9 * n_vs

# 8. Vérification de la disponibilité des NVS
if len(nvs_data) < n_nvs_target:
    raise ValueError(f"Pas assez d'articles NVS avec thématique SA/SV : il faut {n_nvs_target}, or on n'en a que {len(nvs_data)}.")

# 9. Échantillonnage des NVS nécessaires
nvs_sampled = nvs_data.sample(n=n_nvs_target, random_state=42)

# 10. Fusion finale
final_data = pd.concat([vs_data, nvs_sampled], ignore_index=True)

# 11. Vérification des proportions
print(final_data["type_article"].value_counts(normalize=True).round(2))
print(final_data["thematique"].value_counts())


type_article
NVS    0.9
VS     0.1
Name: proportion, dtype: float64
thematique
SA    1253
SV    1237
Name: count, dtype: int64


In [170]:
final_data.head()

,text,type_article,thematique
0,Microbial Community Composition Associated wit...,VS,SV
1,Plant Pathogenic and Endophytic Colletotrichum...,VS,SV
2,Lethal Bronzing: What you should know about th...,VS,SV
3,Leaffooted Bug Damage in Almond Orchards Leaff...,VS,SV
4,Kebbi govt battles mysterious disease affectin...,VS,SV


In [166]:
final_data.shape

(2490, 3)

In [69]:
final_data["type_article"].value_counts()

type_article
NVS    2241
VS      249
Name: count, dtype: int64

In [70]:
final_data["thematique"].value_counts()

thematique
SA    1253
SV    1237
Name: count, dtype: int64

Nous avons avec ce script sauvegardé les deux premières versions des données  

In [171]:
final_data.to_csv("data/data_v2.csv", index=False)

#### Pré-nettoyage

In [172]:
NOMS_MALADIE = [
    "fusarium oxysporum tropical", "hlb", "xylella fastidiosa", "dengue",
    "african swine fever", "avian influenza", "bluetongue",
    "classical swine fever", "foot and mouth disease", "lumpy skin disease",
    "peste des petits ruminants", "schmallengberg virus", "chikungunya", "cholera",
    "coronavirus", "cwd", "disease", "distemper", "echinococcus", "eee", "ehv",
    "fungus", "legionella", "mad cow", "newcastle", "parvovirus", "rabies",
    "rhd", "schmallenberg virus", "suspicion", "unknown",
    "vesicular stomatitis", "west nile", "zika"
]

MALADIE_PATTERN = re.compile("|".join(map(re.escape, NOMS_MALADIE)), flags=re.IGNORECASE)
URL_PATTERN = re.compile(r'http\S+|www\S+|<a .*?>|</a>|<.*?>', flags=re.IGNORECASE)


def supprimer_maladie(texte):
    if not isinstance(texte, str):
        return ""
    texte = URL_PATTERN.sub(" ", texte)
    texte = MALADIE_PATTERN.sub(" ", texte)
    return texte


#### Fonction de nettoyage normal des textes de manière générale 

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation

# Initialisation
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
custom_stopwords = {'et', 'al'} # Les expressions que nous avons retrouvé en grande quantité lors de nos expérimentations et nous avons décidé de ne pas les conserver 


def nettoyer_texte(texte):
    """
    Nettoie un texte brut :
    2. Tokenisation
    3. Nettoyage caractères spéciaux
    4. Suppression stopwords, ponctuation, custom_stopwords
    5. Lemmatisation
    """
    
    # Étape 2 : tokenisation
    tokens = word_tokenize(texte)

    # Étape 3-6 : nettoyage
    tokens_nettoyes = []
    for token in tokens:
        token = token.lower()
        token = re.sub(r'\s+', '', token)
        token = re.sub(r'[^a-zàâçéèêëîïôûùüÿñæœ]', '', token)

        if (
            token
            and token not in stop_words
            and token not in punctuation
            and token not in custom_stopwords
        ):
            token = lemmatizer.lemmatize(token)
            tokens_nettoyes.append(token)

    return ' '.join(tokens_nettoyes)


In [180]:
## data_v1 qu'on va utiliser pour former data_v3

data_v1 = pd.read_csv("data/data_v1.csv")

# On fait une copie de data_v1
data_v3 = data_v1.copy()

In [181]:
# Supression des noms de maladie 
data_v3["text_clean"] = data_v3["text"].apply(supprimer_maladie)

A ce niveau on vérifie car si on vérifie après on ne peut pas être sur que le terme capturé est un nom de maladie étant donné la tokenisation, lemmatisation ...

In [182]:
import re

# Pattern insensible à la casse
pattern = re.compile("|".join(map(re.escape, NOMS_MALADIE)), flags=re.IGNORECASE)

# Vérifions si chaque ligne contient un nom de maladie
maladies_presentes = data_v3["text_clean"].str.contains(pattern, regex=True)

# Combien de lignes en contiennent encore ?
print("Nombre de lignes contenant encore une maladie :", maladies_presentes.sum())


Nombre de lignes contenant encore une maladie : 0


On applique maintenant le nettoyage normal

In [184]:
# Application

# Nettoyage des textes
data_v3["text_clean"] = data_v3["text_clean"].apply(nettoyer_texte)


In [185]:
# On sauvegarde data_v3 qui est la version complètement sans les noms de maladie 

data_v3.to_csv("data/data_v3.csv",index=False)

#### Deuxième version de nettoyage sur les versions 1 et 2 des données 

On applique simplement le nettoyage des textes sans suppression des noms de maladie

In [186]:
data_v1 = pd.read_csv("data/data_v1.csv")

data_v1["text_clean"] = data_v1["text"].apply(nettoyer_texte)

In [188]:
data_v2 = pd.read_csv("data/data_v2.csv")

data_v2["text_clean"] = data_v2["text"].apply(nettoyer_texte)

In [189]:
# On sauvegarde ces versions de données avec la colonne text_clean ajoutée qu'on doit utiliser

data_v1.to_csv("data/data_v1.csv", index=False)
data_v2.to_csv("data/data_v2.csv", index=False)

In [191]:
data_v2.head()

,text,type_article,thematique,text_clean
0,Microbial Community Composition Associated wit...,VS,SV,microbial community composition associated pot...
1,Plant Pathogenic and Endophytic Colletotrichum...,VS,SV,plant pathogenic endophytic colletotrichum fru...
2,Lethal Bronzing: What you should know about th...,VS,SV,lethal bronzing know disease turn palm tree br...
3,Leaffooted Bug Damage in Almond Orchards Leaff...,VS,SV,leaffooted bug damage almond orchard leaffoote...
4,Kebbi govt battles mysterious disease affectin...,VS,SV,kebbi govt battle mysterious disease affecting...


### Reprendre le rassemblage des données avec les différentes versions comme précédemment mais cette fois en prennat tous les articles de la phase 1 étant donné que c'est déjà classé comme NVS, nous allons avoir une très bonne représentation de la réalité, nous prendrons uniqmement les articles de thématique SA et SV 

Nous allons:
- Prendre tous les articles de la phase 1 dans lesquels on n'avait pas supprimé les noms de maladie (clean_text) puis associer aux articles de la phase 2 toujour sans suppression des noms de maladie : data_v1.csv 
- Prendre tous les articles de la phase 1 dans lesquels on avait supprimé les noms de maladie (text_concat) puis associer aux articles de la phase 2 mais ici sans suppression des noms de maladie : data_v2.csv
- Prendre data_v1.csv et supprimer les noms de maladie pour former data_v3.csv 

In [78]:
import pandas as pd

#keep_default_na=False permet juste de dire à pandas de considérer les valeurs NAN comme telle, pas de les prendre comme des 
# chaines de caractère ou autre qui pourrait causer certaines erreurs

data1 = pd.read_csv("data/articles_selectionnes_umap_et_kmeans.csv", keep_default_na=False)
data2 = pd.read_csv("data/articles_SA&SV.csv", keep_default_na=False)




In [27]:
data1.columns

Index(['title', 'text', 'thematique', 'Maladie', 'text_sans_html',
       'text_concat', 'clean_text', 'x', 'y', 'z', 'cluster', 'dist'],
      dtype='object')

In [6]:
data2.columns

Index(['url', 'title', 'text', 'type_article', 'thematique', 'pertinent',
       'word_count'],
      dtype='object')

### data_v1.csv 

Prendre *text+title* de data1 et *text_concat* de data2 pour avoir les articles avec les noms de maladie dans l'ensemble 

In [79]:
## On concatène titre et text des articles de la phase 2, car pour la phase 1 c'est déjà dans la colonne "text_concat"
# On renomme "text_concat" à "text"

data2["text_concat"] = data2["title"] + " " + data2["text"]

data2 = data2[["text_concat", "type_article", "thematique"]]

data2.rename(columns={"text_concat":"text"}, inplace=True)


data2.head()


,text,type_article,thematique
0,Utilizing Environmentally Friendly Techniques ...,NVS,SV
1,Incompatible Browser!\nLooks like you're usin...,NVS,SV
2,Diseased ash trees in Redditch to be felled to...,NVS,SV
3,ScienceDirect There was a problem providing th...,NVS,SV
4,The owner of this website (www.michiganfarmne...,NVS,SV


In [80]:
# On récupère les colonnes qui nous interressent du jeu de données de la phase 1
# On renomme "text_concat" à "text"

data1 = data1[["text_concat","thematique"]]

data1.rename(columns={"text_concat": "text"}, inplace=True)

Rassemblage des deux jeux de données

In [81]:
import pandas as pd

# 1. Marquer les anciennes données (phase 1) comme NVS
data1["type_article"] = "NVS"

# 2. Colonnes communes à conserver
columns = ["text", "type_article", "thematique"]

# 3. Harmonisation des colonnes
data1 = data1[columns]
data2 = data2[columns]

# 4. Fusion complète en évitant les doublons 
full_data = pd.concat([data1, data2], ignore_index=True)
full_data = full_data.drop_duplicates(subset=["text", "thematique"])

# 5. Filtrage uniquement sur les thématiques SA ou SV
filtered_data = full_data[full_data["thematique"].isin(["SA", "SV"])].copy()

# 6. Statistiques générales
print(filtered_data["type_article"].value_counts())
print(filtered_data["type_article"].value_counts(normalize=True).round(2))
print(filtered_data["thematique"].value_counts())


type_article
NVS    16653
VS       246
Name: count, dtype: int64
type_article
NVS    0.99
VS     0.01
Name: proportion, dtype: float64
thematique
SV    8580
SA    8319
Name: count, dtype: int64


In [83]:
filtered_data.to_csv("data/data_v1_phase2_private.csv", index=False)

### data_v2.csv 

Nous prenons la colonne *text_clean* de data1 (déjà concatené et les noms de maladie supprimés) avec *title+text* de data2

Nous avons pensé à faire ceci car c'est l'approche qui nous avait donné de meilleurs scores avant qu'on ne pense à une possibilité de supprimer les noms de maladie dans cette phase 2

In [84]:
# On reprend le chargement afin déviter toute confusion 

#keep_default_na=False permet juste de dire à pandas de considérer les valeurs NAN comme telle, pas de les prendre comme des 
# chaines de caractère ou autre qui pourrait causer certaines erreurs

import pandas as pd 

data1 = pd.read_csv("data/articles_selectionnes_umap_et_kmeans.csv", keep_default_na=False)
data2 = pd.read_csv("data/articles_SA&SV.csv", keep_default_na=False)




In [85]:
## On forme d'abord notre jeu de données de la phase 2 avec les colonnes qui nous interressent 

data2["text_concat"] = data2["title"] + " " + data2["text"]

data2 = data2[["text_concat", "type_article", "thematique"]]

data2.rename(columns={"text_concat":"text"}, inplace=True)


data2.head()


,text,type_article,thematique
0,Utilizing Environmentally Friendly Techniques ...,NVS,SV
1,Incompatible Browser!\nLooks like you're usin...,NVS,SV
2,Diseased ash trees in Redditch to be felled to...,NVS,SV
3,ScienceDirect There was a problem providing th...,NVS,SV
4,The owner of this website (www.michiganfarmne...,NVS,SV


In [86]:
# On récupère les colonnes qui nous interressent du jeu de données de la phase 1

data1 = data1[["clean_text","thematique"]]

data1.rename(columns={"clean_text": "text"}, inplace=True)

In [87]:
import pandas as pd

# 1. Marquer les anciennes données (phase 1) comme NVS
data1["type_article"] = "NVS"

# 2. Colonnes communes à conserver
columns = ["text", "type_article", "thematique"]

# 3. Harmonisation des colonnes
data1 = data1[columns]
data2 = data2[columns]

# 4. Fusion complète en évitant les doublons 
full_data = pd.concat([data1, data2], ignore_index=True)
full_data = full_data.drop_duplicates(subset=["text", "thematique"]) 

# 5. Filtrage uniquement sur les thématiques SA ou SV
filtered_data = full_data[full_data["thematique"].isin(["SA", "SV"])].copy()

# 6. Statistiques générales
print(filtered_data["type_article"].value_counts())
print(filtered_data["type_article"].value_counts(normalize=True).round(2))
print(filtered_data["thematique"].value_counts())


type_article
NVS    16653
VS       247
Name: count, dtype: int64
type_article
NVS    0.99
VS     0.01
Name: proportion, dtype: float64
thematique
SV    8581
SA    8319
Name: count, dtype: int64


### Vérifions qu'il n'y a effectivement pas de doublons 

In [89]:
duplicated_rows = full_data.duplicated(subset=["text", "thematique"], keep=False)
print(full_data[duplicated_rows])


Empty DataFrame
Columns: [text, type_article, thematique]
Index: []


In [90]:
filtered_data.to_csv("data/data_v2_phase2_private.csv", index=False)

### data_v3.csv

Nous allons prendre les donnés de data_v1.csv et nous allons appliquer la fonction de pré-nettoyage qui consistait à supprimer simplement les noms de maladie  

In [91]:
data_v3 = pd.read_csv("data/data_v1_phase2_private.csv")

In [92]:
import re 

NOMS_MALADIE = [
    "fusarium oxysporum tropical", "hlb", "xylella fastidiosa", "dengue",
    "african swine fever", "avian influenza", "bluetongue",
    "classical swine fever", "foot and mouth disease", "lumpy skin disease",
    "peste des petits ruminants", "schmallengberg virus", "chikungunya", "cholera",
    "coronavirus", "cwd", "disease", "distemper", "echinococcus", "eee", "ehv",
    "fungus", "legionella", "mad cow", "newcastle", "parvovirus", "rabies",
    "rhd", "schmallenberg virus", "suspicion", "unknown",
    "vesicular stomatitis", "west nile", "zika"
]

MALADIE_PATTERN = re.compile("|".join(map(re.escape, NOMS_MALADIE)), flags=re.IGNORECASE)
URL_PATTERN = re.compile(r'http\S+|www\S+|<a .*?>|</a>|<.*?>', flags=re.IGNORECASE)


def supprimer_maladie(texte):
    if not isinstance(texte, str):
        return ""
    texte = URL_PATTERN.sub(" ", texte)
    texte = MALADIE_PATTERN.sub(" ", texte)
    return texte


In [93]:
data_v3["text"] = data_v3["text"].apply(supprimer_maladie)

In [94]:
## On peut tester que la suppression a marché ici


# Pattern insensible à la casse
pattern = re.compile("|".join(map(re.escape, NOMS_MALADIE)), flags=re.IGNORECASE)

# Vérifions si chaque ligne contient un nom de maladie
maladies_presentes = data_v3["text"].str.contains(pattern, regex=True)

# Combien de lignes en contiennent encore ?
print("Nombre de lignes contenant encore une maladie :", maladies_presentes.sum())


Nombre de lignes contenant encore une maladie : 0


In [95]:
## Vérifions encore s'il y a des doublons ou pas 

duplicated_rows = data_v3.duplicated(subset=["text", "thematique"], keep=False)
print(data_v3[duplicated_rows])


Empty DataFrame
Columns: [text, type_article, thematique]
Index: []


In [96]:
## Conservons maintenant cette troisième version 

data_v3.to_csv("data/data_v3_phase2_private.csv", index=False)

#### Nous avons maintenant nos 3 versions complètes des données (comportant tous les articles de SA et SV qui sont considérés comme NVS de base)

In [97]:
data_v3.head()

,text,type_article,thematique
0,Bani Suef emergency in preparation for spring ...,NVS,SA
1,Preventive vaccination campaign against foot-a...,NVS,SA
2,Janati: The herd's condition is good. And Onsa...,NVS,SA
3,As Eid al-Adha approaches. Onsa manager reveal...,NVS,SA
4,Intensive clean-up campaigns in assiut neighbo...,NVS,SA


### Appliquons maintenant le nettoyage sur les textes des 3 versions de jeu de données 

- Tokenisation
- Suppression des stop-word
- Suppresion des caractères incompréhensibles
- Mise en minuscle
- Lemmatisation

In [98]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation

# Initialisation
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
custom_stopwords = {'et', 'al'} # Les expressions que nous avons retrouvées en grande quantité lors de nos expérimentations et nous avons décidé de ne pas les conserver 


def nettoyer_texte(texte):
    """
    Nettoie un texte brut :
    2. Tokenisation
    3. Nettoyage caractères spéciaux
    4. Suppression stopwords, ponctuation, custom_stopwords
    5. Lemmatisation
    """
    
    # Étape 2 : tokenisation
    tokens = word_tokenize(texte)

    # Étape 3-6 : nettoyage
    tokens_nettoyes = []
    for token in tokens:
        token = token.lower()
        token = re.sub(r'\s+', '', token)
        token = re.sub(r'[^a-zàâçéèêëîïôûùüÿñæœ]', '', token)

        if (
            token
            and token not in stop_words
            and token not in punctuation
            and token not in custom_stopwords
        ):
            token = lemmatizer.lemmatize(token)
            tokens_nettoyes.append(token)

    return tokens_nettoyes # On retourne d'abord les tokens afin de pouvoir visualiser le wordcloud des articles 

# Ensuite on fera la jointures des tokens pour former les textes nettoyés 


Appliquons cela sur les trois jeux de données 

In [99]:
data_v1 = pd.read_csv("data/data_v1_phase2_private.csv")

data_v2 = pd.read_csv("data/data_v2_phase2_private.csv")

data_v3 = pd.read_csv("data/data_v3_phase2_private.csv")


# Application du nettoyage 
data_v1["tokens_nettoyes"] = data_v1["text"].apply(nettoyer_texte)

data_v2["tokens_nettoyes"] = data_v2["text"].apply(nettoyer_texte)

data_v3["tokens_nettoyes"] = data_v3["text"].apply(nettoyer_texte)

In [100]:
## On ajoute directement la jointures des tokens afin de former les textes nettoyés finaux que nous allons utiliser 

data_v1["text_clean"] = data_v1["tokens_nettoyes"].apply(lambda tokens: " ".join(tokens))

data_v2["text_clean"] = data_v2["tokens_nettoyes"].apply(lambda tokens: " ".join(tokens))

data_v3["text_clean"] = data_v3["tokens_nettoyes"].apply(lambda tokens: " ".join(tokens))


Nous avons maintenant tout ce qu'il nous faut pour commencer les classifications 

In [101]:
## On sauvegarde à nouveau 

data_v1.to_csv("data/data_v1_phase2_private.csv", index=False)

data_v2.to_csv("data/data_v2_phase2_private.csv", index=False)

data_v3.to_csv("data/data_v3_phase2_private.csv", index=False)

#### Sauvegardons déjà pour dataverse

- Version qu'on va garder en restriction: ici contenant les textes originaux on a déjà nommé avec *private*

- Version qu'on pourra publier : contenant uniquement text_clean, type_article et thematique on nommera avec *public*

In [102]:
data_v1 = data_v1[["text_clean", "type_article", "thematique"]]

data_v2 = data_v2[["text_clean", "type_article", "thematique"]]

data_v3 = data_v3[["text_clean", "type_article", "thematique"]]

# sauvegarde 

data_v1.to_csv("data/data_v1_phase2_public.csv", index=False)

data_v2.to_csv("data/data_v2_phase2_public.csv", index=False)

data_v3.to_csv("data/data_v3_phase2_public.csv", index=False)

- Seulement ce qu'on a collecté à la phase 2 sans les articles de la phase 1: data_phase2_private.csv 

data

In [103]:
data_v1.head()

,text_clean,type_article,thematique
0,ban suef emergency preparation spring summer b...,NVS,SA
1,preventive vaccination campaign footandmouth d...,NVS,SA
2,janati herd condition good onsa interacts jett...,NVS,SA
3,eid aladha approach onsa manager reveals moroc...,NVS,SA
4,intensive cleanup campaign assiut neighborhood...,NVS,SA


In [104]:
data_v2.head()

,text_clean,type_article,thematique
0,ban suef emergency preparation spring summer b...,NVS,SA
1,preventive vaccination campaign footandmouth t...,NVS,SA
2,janati herd condition good onsa interacts jett...,NVS,SA
3,eid aladha approach onsa manager reveals moroc...,NVS,SA
4,intensive cleanup campaign assiut neighborhood...,NVS,SA


In [105]:
data_v3.head()

,text_clean,type_article,thematique
0,ban suef emergency preparation spring summer b...,NVS,SA
1,preventive vaccination campaign footandmouth t...,NVS,SA
2,janati herd condition good onsa interacts jett...,NVS,SA
3,eid aladha approach onsa manager reveals moroc...,NVS,SA
4,intensive cleanup campaign assiut neighborhood...,NVS,SA


### Nous allons dans la suite former les données que nous allons sauvegarder pour chaque phase de ce projet

Pour la phase 1, nous devons sauvegarder :
- Les données originales, à partir desquelles nous avons fait la sélection dans le but de faire l'équilibre entre les thématiques: ces données seront privées 
- Les données équilibrées via UMAP & KMEANS : Nous aurons deux versions (publique et privée que nous allons former)
- Les données équilibrées via les noms de maladies évoqués (publique et privée aussi)

Pour la phase 2, nous devons sauvegarder : 
- Les données que nous avons formées à la phase 2 sans les ajouts de la première phase (publique et privée aussi)
- Les données formées en ajoutant quelques données de la phase 1 (publique et privée assi)

#### Commençons par celles de la phase 2 vu que c'est actuellement ce que nous avons sous la main dans notre repertoire 

In [6]:
import pandas as pd 

# Les données formées essentiellement à travers les sources de cette deuxième phase 
data1 = pd.read_csv("data/articles_SA&SV.csv", keep_default_na=False)

data1.head()


,url,title,text,type_article,thematique,pertinent,word_count
0,https://www.mdpi.com/2073-4395/15/7/1551,Utilizing Environmentally Friendly Techniques ...,The review aims to provide a comprehensive ana...,NVS,SV,False,10956
1,https://gns3.com/community/discussions/the-hid...,,Incompatible Browser!\nLooks like you're using...,NVS,SV,False,19
2,https://www.bbc.co.uk/news/articles/cy9xv90lg7do,Diseased ash trees in Redditch to be felled to...,Trees infected with a deadly disease in Reddit...,NVS,SV,False,107
3,https://www.sciencedirect.com/science/article/...,ScienceDirect,There was a problem providing the content you ...,NVS,SV,False,29
4,https://www.michiganfarmnews.com/downy-mildew-...,,The owner of this website (www.michiganfarmnew...,NVS,SV,False,45


In [7]:
# Mettons au propre, enlevons les attributs inutiles 

data1 = data1[["url", "title", "text", "type_article", "thematique", "word_count"]]

data1.head()

,url,title,text,type_article,thematique,word_count
0,https://www.mdpi.com/2073-4395/15/7/1551,Utilizing Environmentally Friendly Techniques ...,The review aims to provide a comprehensive ana...,NVS,SV,10956
1,https://gns3.com/community/discussions/the-hid...,,Incompatible Browser!\nLooks like you're using...,NVS,SV,19
2,https://www.bbc.co.uk/news/articles/cy9xv90lg7do,Diseased ash trees in Redditch to be felled to...,Trees infected with a deadly disease in Reddit...,NVS,SV,107
3,https://www.sciencedirect.com/science/article/...,ScienceDirect,There was a problem providing the content you ...,NVS,SV,29
4,https://www.michiganfarmnews.com/downy-mildew-...,,The owner of this website (www.michiganfarmnew...,NVS,SV,45


In [8]:
data1.to_csv("data/data_phase2_without_data_phase1_private.csv", index=False)

In [9]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation

# Initialisation
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
custom_stopwords = {'et', 'al'} # Les expressions que nous avons retrouvé en grande quantité lors de nos expérimentations et nous avons décidé de ne pas les conserver 


def nettoyer_texte(texte):
    """
    Nettoie un texte brut :
    2. Tokenisation
    3. Nettoyage caractères spéciaux
    4. Suppression stopwords, ponctuation, custom_stopwords
    5. Lemmatisation
    """
    
    # Étape 2 : tokenisation
    tokens = word_tokenize(texte)

    # Étape 3-6 : nettoyage
    tokens_nettoyes = []
    for token in tokens:
        token = token.lower()
        token = re.sub(r'\s+', '', token)
        token = re.sub(r'[^a-zàâçéèêëîïôûùüÿñæœ]', '', token)

        if (
            token
            and token not in stop_words
            and token not in punctuation
            and token not in custom_stopwords
        ):
            token = lemmatizer.lemmatize(token)
            tokens_nettoyes.append(token)

    return ' '.join(tokens_nettoyes)


In [10]:
## Nous allons nettoyer les textes, et former un jeu de données avec les textes nettoyés et juste les colonnes type_article
# et thematique 

data1["text_concat"] = data1["title"] + " " + data1["text"]

data1["text_clean"] = data1["text_concat"].apply(nettoyer_texte)

In [11]:
data1.head()

,url,title,text,type_article,thematique,word_count,text_concat,text_clean
0,https://www.mdpi.com/2073-4395/15/7/1551,Utilizing Environmentally Friendly Techniques ...,The review aims to provide a comprehensive ana...,NVS,SV,10956,Utilizing Environmentally Friendly Techniques ...,utilizing environmentally friendly technique s...
1,https://gns3.com/community/discussions/the-hid...,,Incompatible Browser!\nLooks like you're using...,NVS,SV,19,Incompatible Browser!\nLooks like you're usin...,incompatible browser look like using older bro...
2,https://www.bbc.co.uk/news/articles/cy9xv90lg7do,Diseased ash trees in Redditch to be felled to...,Trees infected with a deadly disease in Reddit...,NVS,SV,107,Diseased ash trees in Redditch to be felled to...,diseased ash tree redditch felled protect resi...
3,https://www.sciencedirect.com/science/article/...,ScienceDirect,There was a problem providing the content you ...,NVS,SV,29,ScienceDirect There was a problem providing th...,sciencedirect problem providing content reques...
4,https://www.michiganfarmnews.com/downy-mildew-...,,The owner of this website (www.michiganfarmnew...,NVS,SV,45,The owner of this website (www.michiganfarmne...,owner website wwwmichiganfarmnewscom banned co...


In [12]:
data1 = data1[["text_clean", "type_article", "thematique"]]

In [13]:
data1.to_csv("data/data_phase2_without_data_phase1_public.csv", index=False)

Nous formons maintenant les données ajoutées à partir de celles de la phase 1 

In [14]:
data2 = pd.read_csv("data/data_classification_2.csv")

data2.head()

,text,tokens_clean,text_clean,type_article,thematique
0,Microbial Community Composition Associated wit...,"['microbial', 'community', 'composition', 'ass...",microbial community composition associated pot...,VS,SV
1,Plant Pathogenic and Endophytic Colletotrichum...,"['plant', 'pathogenic', 'endophytic', 'colleto...",plant pathogenic endophytic colletotrichum fru...,VS,SV
2,Lethal Bronzing: What you should know about th...,"['lethal', 'bronzing', 'know', 'disease', 'tur...",lethal bronzing know disease turn palm tree br...,VS,SV
3,Leaffooted Bug Damage in Almond Orchards Leaff...,"['leaffooted', 'bug', 'damage', 'almond', 'orc...",leaffooted bug damage almond orchard leaffoote...,VS,SV
4,Kebbi govt battles mysterious disease affectin...,"['kebbi', 'govt', 'battle', 'mysterious', 'dis...",kebbi govt battle mysterious disease affecting...,VS,SV


In [15]:
## On peut sauvegarder la version privée directement à partir de ceci 

data2.to_csv("data/data_final_phase2_private.csv", index=False)

In [ ]:
## Sélectionnons les colonnes text_clean, type_article et thematique pour former la version publique 

data2 = data2[["text_clean", "type_article", "thematique"]]

# Renommons text_clean en text tout simplement 

data2.rename(columns={"text_clean": "text"}, inplace=True)

data2.to_csv("data/data_final_phase2_public.csv", index=False)

Nous avons déjà les données de la phase 2

### Suite

Formons maintenant les données de la phase 1

In [1]:
import pandas as pd 

data1 = pd.read_csv("data/articles_selectionnes_umap_et_kmeans.csv")

data1.head()

,title,text,thematique,Maladie,text_sans_html,text_concat,clean_text,x,y,z,cluster,dist
0,Bani Suef emergency in preparation for spring ...,Bani Suef emergency in preparation for spring ...,SA,Foot And Mouth Disease,Bani Suef emergency in preparation for spring ...,Bani Suef emergency in preparation for spring ...,Bani Suef emergency in preparation for spring ...,8.586625,0.908350,1.933475,0.0,NaN
1,Preventive vaccination campaign against foot-a...,Preventive vaccination campaign against foot-a...,SA,Foot And Mouth Disease,Preventive vaccination campaign against foot-a...,Preventive vaccination campaign against foot-a...,Preventive vaccination campaign against foot-a...,8.597440,0.935446,1.968340,0.0,NaN
2,Janati: The herd's condition is good. And Onsa...,Janati: The herd's condition is good. And Onsa...,SA,Foot And Mouth Disease,Janati: The herd's condition is good. And Onsa...,Janati: The herd's condition is good. And Onsa...,Janati: The herd's condition is good. And Onsa...,8.555155,0.879633,1.868251,0.0,NaN
3,As Eid al-Adha approaches. Onsa manager reveal...,2020-07-21\n\nIn Arabic LeSiteinfo - Wadih Taw...,SA,Foot And Mouth Disease,2020-07-21\n\nIn Arabic LeSiteinfo - Wadih Taw...,As Eid al-Adha approaches. Onsa manager reveal...,As Eid al-Adha approaches. Onsa manager reveal...,8.442227,0.915807,1.994216,0.0,NaN
4,Intensive clean-up campaigns in assiut neighbo...,Asyut governorate witnessed the continuation o...,SA,Foot And Mouth Disease,Asyut governorate witnessed the continuation o...,Intensive clean-up campaigns in assiut neighbo...,Intensive clean-up campaigns in assiut neighbo...,8.603223,0.973532,1.883030,0.0,NaN


In [3]:
## Formons la version privée 

data1 = data1[["title", "text", "thematique", "Maladie", "text_sans_html", "clean_text"]]

## Sauvegarde 
data1.to_csv("data/data_selected_with_umap&kmeans_phase1_private.csv", index=False)

In [4]:
### Nous y prenons uniquement les textes nettoyés, la thématique et nom de maladie : nous allons renommer clean_text en text 

data1 = data1[["clean_text", "thematique", "Maladie"]]

data1.rename(columns={"clean_text": "text"}, inplace=True)

data1.to_csv("data/data_selected_with_umap&kmeans_phase1_public.csv", index=False)

In [5]:
## Les données sélectionnées par maladie 
import pandas as pd 

data1 = pd.read_csv("data/articles_selectionnes_par_maladie.csv")

data1.head()

,title,text,Thematique,Maladie,text_sans_html,texte_concat,clean_text
0,Google News,"Google News\n\n<a href=""https://news.google.co...",SV,Fusarium Oxysporum Tropical,Google News\n\nINIFAP develops technology to p...,Google News Google News\n\nINIFAP develops tec...,Google News Google News\n\nINIFAP develops tec...
1,Investigating the Dynamics of Bayoud Disease i...,Investigating the Dynamics of Bayoud Disease i...,SV,Fusarium Oxysporum Tropical,Investigating the Dynamics of Bayoud Disease i...,Investigating the Dynamics of Bayoud Disease i...,Investigating the Dynamics of Bayoud in Date ...
2,Google News,"Google News\n\n<a href=""https://news.google.co...",SV,Fusarium Oxysporum Tropical,Google News\n\nAnte riesgo de hongo “devastado...,Google News Google News\n\nAnte riesgo de hong...,Google News Google News\n\nAnte riesgo de hong...
3,PHL banana exports up 3.4% amid efforts to cur...,PHL banana exports up 3.4% amid efforts to cur...,SV,Fusarium Oxysporum Tropical,PHL banana exports up 3.4% amid efforts to cur...,PHL banana exports up 3.4% amid efforts to cur...,PHL banana exports up 3.4% amid efforts to cur...
4,"Research points to ways out against the ""banan...","Research points to ways out against the ""banan...",SV,Fusarium Oxysporum Tropical,"Research points to ways out against the ""banan...","Research points to ways out against the ""banan...","Research points to ways out against the ""banan..."


In [6]:
## Formons la version privée 

data1 = data1[["title", "text", "Thematique", "Maladie", "text_sans_html", "clean_text"]]

## Sauvegarde 
data1.to_csv("data/data_selected_by_disease_phase1_private.csv", index=False)

In [7]:
### Nous y prenons uniquement les textes nettoyés, la thématique et nom de maladie : nous allons renommer clean_text en text 

data1 = data1[["clean_text", "Thematique", "Maladie"]]

data1.rename(columns={"clean_text": "text"}, inplace=True)

data1.to_csv("data/data_selected_by_disease_phase1_public.csv", index=False)

Les données originales de la phase1 à partir desquelles nous avons fait la sélection pour équilibrer les articles par thématique 

In [12]:
data = pd.read_csv("data/articles_pretraites_complets_phase1.csv")

data.head()

,title,text,Thematique,Maladie,text_sans_html,texte_concat,clean_text
0,Federal Government confirms need for hunting,Federal Government confirms need for hunting \...,SA,African Swine Fever,Federal Government confirms need for hunting \...,Federal Government confirms need for hunting F...,Federal Government confirms need for hunting F...
1,ASP new infections in Poland not abated,ASP new infections in Poland not abated \nCont...,SA,African Swine Fever,ASP new infections in Poland not abated \nCont...,ASP new infections in Poland not abated ASP ne...,ASP new infections in Poland not abated ASP ne...
2,SPANISH CUCUMBERS are not a SOURCE of infectio...,SPANISH CUCUMBERS are not a SOURCE of infectio...,SA,African Swine Fever,SPANISH CUCUMBERS are not a SOURCE of infectio...,SPANISH CUCUMBERS are not a SOURCE of infectio...,SPANISH CUCUMBERS are not a SOURCE of infectio...
3,African Swine Fever: Recent Developments and T...,African Swine Fever: Recent Developments and T...,SA,African Swine Fever,African Swine Fever: Recent Developments and T...,African Swine Fever: Recent Developments and T...,: Recent Developments and Timely Updates : Rec...
4,African Swine Fever: Recent Developments and T...,African Swine Fever: Recent Developments and T...,SA,African Swine Fever,African Swine Fever: Recent Developments and T...,African Swine Fever: Recent Developments and T...,: Recent Developments and Timely Updates - The...


In [9]:
data["Thematique"].value_counts()

Thematique
SA    93398
SP     8819
SV     8168
Name: count, dtype: int64

In [13]:
## Formons la version privée de ces données 
data = data[["title", "text", "clean_text", "text_sans_html", "Thematique", "Maladie"]]

data.to_csv("data/data_complete_private_phase1.csv", index=False)

In [14]:
## Formons la version privée 

data = data[["clean_text", "Thematique", "Maladie"]]

# Renommer clean_text en text
data.rename(columns={"clean_text":"text"}, inplace=True)
data.to_csv("data/data_complete_public_phase1.csv", index=False)